In [1]:
import os
import logging
import json

from dotenv import load_dotenv
from datetime import datetime

from df_processing import user_df, report_df_processing, datatype_conversion, clean_html

from core.jinja_templating import SQLTemplateLoader
from core.excel_utils import cool_write_function, empty_directory
from core.sftp_management import SFTPManager

In [2]:
# loading .env file 
load_dotenv()
logger = logging.getLogger(__name__)

In [3]:
## adjust this to your user path
user_path = "/Users/blayton"

with open(f"{user_path}/Documents/solution_engineering/cc_reports/cc_asktq_reporting/config.json", 'r') as file:
    data = json.load(file)  

In [4]:
organization_id = data["organization_id"]
customer_abbreviation = data["customer_abbreviation"]
output_directory_base=data["output_directory"]
sql_template_path_base=data["template_directory"]

output_directory=f"{user_path}/{output_directory_base}"
sql_template_path=f"{user_path}/{sql_template_path_base}"

In [5]:
## rendering sql templates using jinja
if __name__ == "__main__":
    sql_loader = SQLTemplateLoader(sql_template_path)

    rendered = sql_loader.render_all_templates(**data)


Loaded 2 SQL templates


In [6]:
def report_generation(**kwargs):
    start_time = datetime.now()

    users_df = user_df(kwargs['users_report'])
    del kwargs['users_report']

    #clears the existing output directory
    empty_directory(output_directory)

    for key, query in kwargs.items():

        if key == 'ask_tq_report':
            df = report_df_processing(users_df, query)

        df = datatype_conversion(df)

        df['response'] = df['response'].apply(clean_html)
        df['response'] = df['response'].str.strip()

        drop_sub_df = df[~df['creator_email'].astype(str).str.contains('@turquoise.health', regex=False, na=False)].reset_index(drop=True)
        df = drop_sub_df

        df.drop(columns=['creator_email'], errors='ignore', inplace=True)

        # Write the resulting dataframe to a workbook
        cool_write_function(customer_abbreviation, output_directory, key, df)
        logger.info(f"{key} component processed.")

    end_time = datetime.now()
    execution_time = end_time - start_time

    logger.info(f"Reports generated: {execution_time} seconds")

In [7]:
def sftp():
    sftp_creds = data['sftp_creds']
    sftp_host = sftp_creds.get('sftp_host')
    sftp_port = sftp_creds.get('sftp_port')
    remote_output = sftp_creds.get('remote_output')
    parent_directory = sftp_creds.get('parent_directory')
    local_path_base = sftp_creds.get('local_path')
    local_path = f"{user_path}/{local_path_base}"

    all_file_paths = []

    # prepping output path
    current_date = datetime.now().strftime('%Y-%m-%d')
    remote_path = os.path.join(remote_output, parent_directory, current_date)
        
    logging.info(f"Traversing directory: {local_path}")
    
    for root, dirs, files in os.walk(local_path):
        for file in files:
            # Create the full path by joining the root and filename
            full_path = os.path.join(root, file)
            
            all_file_paths.append(full_path)

    sftp_username = os.getenv('CSH_SFTP_USERNAME') 
    sftp_password = os.getenv('CSH_SFTP_PASSWORD') 

    sftp_manager = SFTPManager()
    sftp_manager.connect(hostname=sftp_host, port=sftp_port, username=sftp_username, password=sftp_password)

    # Building file path, parent_directory is specific to the type of report based on config.json
    sftp_manager.mkdir_p(remote_path)
            
    for local_file in all_file_paths:
        filename = os.path.basename(local_file)
        file_remote_path = os.path.join(remote_path, filename)

        sftp_manager.upload_file(local_file, file_remote_path)
    
    sftp_manager.disconnect()

In [8]:
report_generation(**rendered)

INFO:core.query_utils.query_utils:Running query for DataFrame: SELECT au.id,
    au.first_name,
    au.last_name,
    au.email,
    ou.department,
    STRING_AGG(CONCAT(org.role, ' - ', op.name), ', ') AS cc_role_permissions,
    ou.role AS user_title,
    oog.name AS group_name,
    au.last_login,
    oogou.organization_group_id
FROM auth_user au
    INNER JOIN tq_organizations_organizationuser ou ON au.id = ou.user_id
    LEFT JOIN tq_organizations_organizationuser_groups ooug ON ooug.organizationuser_id = ou.id
    LEFT JOIN tq_organizations_rolegroup org ON org.group_ptr_id = ooug.group_id
    LEFT JOIN tq_organizations_product op ON op.id = org.product_id
    LEFT JOIN tq_organizations_organizationgrouporganizationuser oogou ON oogou.organization_user_id = ou.id
    LEFT JOIN tq_organizations_organizationgroup oog ON oog.organization_id = ou.organization_id
    AND oog.id = oogou.organization_group_id
WHERE (ou.organization_id = '28'
    AND au.is_staff != 'TRUE'
    AND ou.delete

                                               question  \
0     What are the Medicare Advantage rates in the U...   
1           contracts with a dispute resolution process   
2     reimbursement rates that differ between gold, ...   
3     What exchange/marketplace hospital contracts h...   
4     What exchange/marketplace hospital contracts h...   
...                                                 ...   
1044  what is the For Non-Gatekeeper product revenue...   
1045                        what is the effective date?   
1046                        what is the effective date?   
1047                        what is the effective date?   
1048               What is the payment dispute process?   

                                               response  \
0     <article>    <p>        <strong>The Medicare A...   
1     <article>    <p>        <strong>Several contra...   
2     <article>    <p>        <strong>There are some...   
3     <article>    <p>        Based on reviewing the...

INFO:__main__:ask_tq_report component processed.
INFO:__main__:Reports generated: 0:00:06.870523 seconds


File saved to /Users/blayton/Documents/solution_engineering/cc_reports/cc_asktq_reporting/output/CC_CSH_ASK_TQ_REPORT_20250606.xlsx


In [9]:
sftp()

INFO:root:Traversing directory: /Users/blayton/Documents/solution_engineering/cc_reports/cc_asktq_reporting/output
INFO:paramiko.transport:Connected (version 2.0, client AWS_SFTP_1.2)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [10]:
empty_directory(output_directory)

Deleted file: /Users/blayton/Documents/solution_engineering/cc_reports/cc_asktq_reporting/output/CC_CSH_ASK_TQ_REPORT_20250606.xlsx
